# Construction des dictionnaires pour la Reconnaissance Optique de caractère

Dans ce notebook sera présenté le pré-traitement du jeu de donnée [NIST 2nd](https://www.nist.gov/srd/nist-special-database-19) afin d'obtenir un nouveau jeu de donnée données plus facilement utilisable pour notre algorithme de reconnaissance de caractère

Ce pré-traitement de donnée aura plusieurs objectifs :
- Sélectionner les bonnes données parmis le dataset ( uniquement les lettres majuscules ou minuscules)
- Redimensionner les images pour obtenir des images plus petites afin de soulager notre algorithme d'OCR
- Trier ses données pour obtenir l'arborescence de dataset qui sera décrit ci-dessous

**structure du nouveau dataset obtenu:**
 - NIST-dataset
    - train_set
        - a
            - a000001.png
            - a000002.png
            ...
            ...
            - a00000n.png
        - b ....
        ...
        - z ...
    - test_set
        - a
            - a000001.png
            - a000002.png
            ...
            ...
            - a00000n.png
        - b ....
        ...
        - z ...

    **Pour exécuter le notebook, renommer le dossier "by_class" en "NIST-dataset" puis placer celui-ci le sous dossier data/raw.**

## Importation des ressources nécessaires

### Importation des dépandances.

In [1]:
import os
from glob import glob
from random import shuffle
from string import ascii_lowercase, ascii_uppercase
import pandas as pd

## Déclaration des paramètres

Le nombre d'HSF que l'on veut charger à une influence directe sur le temps de calculs et la quantité de ressources nécessaires pour exécuter ce notebook.
1 HSF représente 1/8 du dataset, donc 4 HSF représenteront 1/2 du dataset

In [2]:
DATASET_INPUT="../../data/raw/NIST-dataset" # Chemin ou l'on va récupérer le datatset
DATASET_OUTPUT="../../data/processed/NIST-dataset" # Chemin ou le nouveau dataset sera stocké
DATASET_CSV_OUTPUT="../../data/pretreated/NIST-dataset/nist.csv"
NUMBER_OF_HSF = [0] # Le nombre de HSF que l'on veut charger
PERCENT_OF_DATA = 0.1
RANDOM=True

## Importation du dataset
### seléction des données voulus
Dans ce présent notebook, nous voulons sélectionner uniquement les lettres a-z et A-Z. Pour ce faire on doit parcourir l'arborescence du dataset, sachant que chaque caractère sera stocké dans un dossier portant comme nom, le nombre ASCII en hexadécimal correspondant.

In [3]:
alphabet_hex = [hex(ord(char)) for char in ascii_lowercase + ascii_uppercase]
alphabet_hex_truncated = [hex_str.split("x")[1] for hex_str in alphabet_hex]

alphabet_hex_truncated[:5]

['61', '62', '63', '64', '65']

## Préparation du dataframe

Le but de ses opérations seront de référencer toutes les données voulues du jeu de donnée dans un dataframe panda qui sera utilisé pour effectuer les divers traitements sur les images

In [4]:
COLUMNS = ["letter", "hsf_number", "old_file_name", "new_file_name","old_path", "new_path"]
dataset = pd.DataFrame(columns=COLUMNS, dtype=str)

## Remplir le dataset
Maintenant que les données souhaités sont connus, nous pouvons commencer à remplir le dataset
### Fonctions utilitaires

Fonction pour récupérer un char depuis une chaine de charactère hexedécimal ascii

In [5]:
def hex_char_to_str(ascii_hex_truncated_char):
    ascii_hex_char = f"0x{ascii_hex_truncated_char}"
    ascii_decimal = int(ascii_hex_char, 16)
    return chr(ascii_decimal)

hex_char_to_str("5a") # Exemple avec la chaine troncaturé ascii de a en hexa

'Z'

Fonction pour récupérer une dictionnaire représentant une ligne du dataset

In [6]:
def get_new_row(letter_value, hsf_number_value, old_file_name_value, new_file_name_value, old_path_value, new_path_value):
    new_row = {COLUMNS[0]: letter_value, COLUMNS[1]: hsf_number_value, COLUMNS[2]: old_file_name_value, COLUMNS[3]: new_file_name_value, COLUMNS[4]: old_path_value, COLUMNS[5]: new_path_value}

    return new_row

Fonction ajouter une nouvelle ligne à la fin du dataset

In [7]:
def dataset_append_row(row):
    dataset.loc[len(dataset)] = row

Fonction pour obtenir tous les fichiers logeant dans un répertoire d' un certain type

In [8]:
def get_files_in_directory(path, file_type):
    search_path = os.path.join(path, f"*.{file_type}")
    list_files = [file for file in glob(search_path)]
    number_of_element = int(len(list_files) * PERCENT_OF_DATA)
    if RANDOM:
        shuffle(list_files)
    return list_files[:number_of_element]
    
get_files_in_directory("../../data/raw/NIST-dataset/4a/hsf_0", "png")[:5] # On teste le bon fonctionnement de la fonction

['../../data/raw/NIST-dataset/4a/hsf_0/hsf_0_00819.png',
 '../../data/raw/NIST-dataset/4a/hsf_0/hsf_0_00646.png',
 '../../data/raw/NIST-dataset/4a/hsf_0/hsf_0_00802.png',
 '../../data/raw/NIST-dataset/4a/hsf_0/hsf_0_00342.png',
 '../../data/raw/NIST-dataset/4a/hsf_0/hsf_0_00321.png']

Fonction pour formater un chiffre de la manière suivant:
1 -> 001

In [9]:
def format_indice(indice):
    return '{0:03}'.format(indice) # 1 -> 001

### Remplissage effectif du dataset en utilisant les fonctions définis précédemment

In [10]:
for ascii_hex_truncated_char in alphabet_hex_truncated:
    for hsf_number in NUMBER_OF_HSF:
        path = os.path.join(DATASET_INPUT, ascii_hex_truncated_char, f"hsf_{hsf_number}")
        char = hex_char_to_str(ascii_hex_truncated_char)
        indice = 0
        print(path) # On observe l'avancement du programme
        for file_path in get_files_in_directory(path, "png"):
            old_file_name = file_path.split(os.sep)[-1]
            new_file_name = f"{char}{format_indice(indice)}.png"
            new_row = get_new_row(char, hsf_number, old_file_name, new_file_name, path, DATASET_OUTPUT)
            dataset_append_row(new_row)
            indice = indice + 1

../../data/raw/NIST-dataset/61/hsf_0
../../data/raw/NIST-dataset/62/hsf_0
../../data/raw/NIST-dataset/63/hsf_0
../../data/raw/NIST-dataset/64/hsf_0
../../data/raw/NIST-dataset/65/hsf_0
../../data/raw/NIST-dataset/66/hsf_0
../../data/raw/NIST-dataset/67/hsf_0
../../data/raw/NIST-dataset/68/hsf_0
../../data/raw/NIST-dataset/69/hsf_0
../../data/raw/NIST-dataset/6a/hsf_0
../../data/raw/NIST-dataset/6b/hsf_0
../../data/raw/NIST-dataset/6c/hsf_0
../../data/raw/NIST-dataset/6d/hsf_0
../../data/raw/NIST-dataset/6e/hsf_0
../../data/raw/NIST-dataset/6f/hsf_0
../../data/raw/NIST-dataset/70/hsf_0
../../data/raw/NIST-dataset/71/hsf_0
../../data/raw/NIST-dataset/72/hsf_0
../../data/raw/NIST-dataset/73/hsf_0
../../data/raw/NIST-dataset/74/hsf_0
../../data/raw/NIST-dataset/75/hsf_0
../../data/raw/NIST-dataset/76/hsf_0
../../data/raw/NIST-dataset/77/hsf_0
../../data/raw/NIST-dataset/78/hsf_0
../../data/raw/NIST-dataset/79/hsf_0
../../data/raw/NIST-dataset/7a/hsf_0
../../data/raw/NIST-dataset/41/hsf_0
.

On visualise le jeu de données obtenu

In [11]:
dataset.head(5)

,letter,hsf_number,old_file_name,new_file_name,old_path,new_path
0,a,0,hsf_0_00165.png,a000.png,../../data/raw/NIST-dataset/61/hsf_0,../../data/processed/NIST-dataset
1,a,0,hsf_0_01232.png,a001.png,../../data/raw/NIST-dataset/61/hsf_0,../../data/processed/NIST-dataset
2,a,0,hsf_0_00029.png,a002.png,../../data/raw/NIST-dataset/61/hsf_0,../../data/processed/NIST-dataset
3,a,0,hsf_0_02687.png,a003.png,../../data/raw/NIST-dataset/61/hsf_0,../../data/processed/NIST-dataset
4,a,0,hsf_0_01890.png,a004.png,../../data/raw/NIST-dataset/61/hsf_0,../../data/processed/NIST-dataset


In [12]:
dataset.tail(5)

,letter,hsf_number,old_file_name,new_file_name,old_path,new_path
12072,Z,0,hsf_0_00160.png,Z036.png,../../data/raw/NIST-dataset/5a/hsf_0,../../data/processed/NIST-dataset
12073,Z,0,hsf_0_00255.png,Z037.png,../../data/raw/NIST-dataset/5a/hsf_0,../../data/processed/NIST-dataset
12074,Z,0,hsf_0_00196.png,Z038.png,../../data/raw/NIST-dataset/5a/hsf_0,../../data/processed/NIST-dataset
12075,Z,0,hsf_0_00284.png,Z039.png,../../data/raw/NIST-dataset/5a/hsf_0,../../data/processed/NIST-dataset
12076,Z,0,hsf_0_00132.png,Z040.png,../../data/raw/NIST-dataset/5a/hsf_0,../../data/processed/NIST-dataset


## Sauvegarde dataset listant les images et leurs noms sous forme de csv.

In [13]:
dataset.to_csv(DATASET_CSV_OUTPUT)